In [1]:
import pandas as pd
import os
from os.path import join
import shutil
import pyodbc

In [2]:
sciezka_from = '../data/raw/'
sciezka_to = '../data/interim/'


In [3]:

sql = """
select pat.Last_Name, pat.First_Name, pat.Pat_ID1, id.ida, id.idc
FROM Patient pat
JOIN ident id ON id.pat_id1 = pat.pat_id1
where id.IDA in (\'{}\')
OR id.IDC in (\'{}\')
"""

In [4]:
def read_credentials_file(file_path):
    credentials = {}
    with open(file_path, 'r') as file:
        for line in file:
            key, value = line.strip().split(':')
            credentials[key.strip()] = value.strip()
    return credentials


def get_sql_list(sql_str, uid, pwd, server, db):
    #Uzyskuje obiekt cursor przez połączenie z serwerem SQL

    conn = pyodbc.connect(
    #Trusted_Connection = 'Yes',
    DRIVER='{SQL Server}',
    Server=server, 
    Database=db,
    UID=uid,
    pwd=pwd
    )
    cursor = conn.cursor()
    fetch = cursor.execute(sql_str).fetchall()

    return fetch

    
file_path = '../../Credentials/Credentials_mosaiq.txt'

credentials_dict = read_credentials_file(file_path)


password = credentials_dict['password']
user = credentials_dict['login']
db = credentials_dict['databasename']
server = credentials_dict['server']


In [5]:
def get_basic(df):
    #Zwraca słownik z parametrami DVH np czy jest % lub w cGy
    basics = df.columns.str.strip()
    params = {x.split(': ')[0]:x.split(': ')[1] for x in basics}
    return params


def csv_to_record(filename):
    df = pd.read_csv(filename
                 , encoding='cp1252'
                 , delimiter='\t'
                 )
    return df


def get_dvh_table(df):
    #Zwraca 3-kolumnową tabelę DVH do dalszych obliczeń
    df.columns
    column_names = ['Structure Name', 'Dose', 'Volume']
    df = df.iloc[1:-1]
    df.columns = df.iloc[0].str.strip()
    df = df[1:]
    df=df[['Structure Name', 'Dose', 'Volume']]
    for i in column_names:
        df[i] = df[i].str.strip()
    for i in column_names[1:]:
        df[i] = df[i].astype(float)
    return df
    
def get_stuct_names(biger_df)->list:
    
    df = get_dvh_table(biger_df)
    #Uzyskaj nazwy struktur
    return df['Structure Name'].unique()


def wyswietl(tekst:str, lista:list):
    print(tekst.capitalize(), f" {len(lista)} pliki:\n")
    for i in lista:
        print(i)
    print("\n")


def get_ptv_name_inny(struct)->tuple:
    find = list()
    for str_name in struct:
        str_name_u=str_name.upper()
        if str_name_u.find('PTV') > -1:
            find.append(str_name)
    return find, len(find)


def get_piers_name(struct)->tuple:
    find=list()
    for str_name in struct:
        str_name_u=str_name.upper()
        if str_name_u.find('PIERS') > -1:
            find.append(str_name)
            
    return find, len(find)


def get_serce_name(struct)->tuple:
    find=list()
    for str_name in struct:
        str_name_u=str_name.upper()
        if str_name_u.find('SERCE') > -1:
            find.append(str_name)
            
    return find, len(find)


def get_pluco_L_name(struct)->tuple:
    find = list()
    for str_name in struct:
        str_name_u=str_name.upper()
        if str_name_u.find('PLUCO') > -1:
            find.append(str_name)
            if len(find)>1:
                for i in find:
                    
                    if i.lower().count('l') == 1:
                        find.remove(i)
    return find, len(find)


def get_pluco_P_name(struct)->tuple:
    find = list()
    for str_name in struct:
        str_name_u=str_name.upper()
        if str_name_u.find('PLUCO') > -1:
            find.append(str_name)
            if len(find)>1:
                for i in find:
                    if i.upper().count('P') == 1:
                        find.remove(i)
    return find, len(find)


def get_external_name(struct)->tuple:
    find = list()
    for str_name in struct:
        str_name_u=str_name.upper()
        if str_name_u.find('EXTERNAL') > -1:
            find.append(str_name)
            if len(find)>1:
                for i in find:
                    if i.upper().count('DIBH') != 1:
                        find.remove(i)
    return find, len(find)


def is_cumulative(df):
    t_df = get_dvh_table(df)
    t_df = t_df.loc[t_df['Structure Name']=='Carbon Fiber']
    t_df = t_df.iloc[0]['Volume']
    return t_df>0 and t_df*0==0
    

### Pętla programu

In [7]:
dose_units = 'Gy'
volume_units = 'cm³'


przeniesiono = list()
odrzucone = {}
for entry in os.scandir(sciezka_from):
    df = csv_to_record(join(sciezka_from,entry.name))
    basic = get_basic(df)

    struct = get_stuct_names(df)
    real_struct = list()
    real_struct.append(get_ptv_name_inny(struct))
    real_struct.append(get_piers_name(struct))
    real_struct.append(get_serce_name(struct))
    real_struct.append(get_pluco_L_name(struct))
    real_struct.append(get_pluco_P_name(struct))
    real_struct.append(get_external_name(struct))
    
    #Sprawdzamy czy jest jedna struktura danego typu
    err_struct = [i for i in real_struct if i[1]>1]
    

    cum = is_cumulative(df)

    

    if basic['Dose Units'] == dose_units and basic['Volume Units'] == volume_units and len(err_struct)==0 and cum:
        przeniesiono.append(f"{entry.name}")
        shutil.move(join(sciezka_from,entry.name), join(sciezka_to,entry.name))
    else:
        if cum:
            cumulative = ''
        else:
            cumulative = 'typ: differential'

        appendix = ''
        if len(err_struct)>0:
            appendix = f'         błąd w strukturach   {err_struct}'

        if basic['Dose Units'] != dose_units or basic['Volume Units'] != volume_units:
            dose = f"dawka jest w {basic['Dose Units']}, a Volume w {basic['Volume Units']}"
        else:
            dose=''

        odrzucone[basic['Patient ID']] = f"{entry.name:<55} {cumulative:<20}{dose:<40}"+appendix

wyswietl('Przeniesiono', przeniesiono)



sql_where_clause = "', '".join(odrzucone)

sql_uni = sql.format(sql_where_clause, sql_where_clause)
pat_names = get_sql_list(sql_uni, user, password, server, db)


def dict_combine(pat_names:list, odrzucone)->list:
    return [f"{i[0]+' '+i[1]:<40} plik: {odrzucone[i[4]]}" for i in pat_names]
        
         
        
odrzucone2 = dict_combine(pat_names, odrzucone)

wyswietl('Orzucono', odrzucone2)

Przeniesiono  1 pliki:

68092300947_DIBHmotylPiersLFFF_DVH_2.csv


Orzucono  3 pliki:

Boryś-Machul Gabriela                    plik: 64011916066_DIBHpiersLmotylFFF1b_DVH_2borys.csv                             dawka jest w Gy, a Volume w %                    błąd w strukturach   [(['PTV 42.5Gy/17fr', 'PTV loza 52.5Gy/21fr'], 2)]
Czaplińska-Głąb Bogusława                plik: 68092300947_DIBHmotylPiersLFFF_DVH_2czaplinska.csv                          dawka jest w Gy, a Volume w %           
Bartosik Agnieszka                       plik: 72053009626_DIBHpiersLFFF2_DVH_2glab.csv                                    dawka jest w Gy, a Volume w %           


